<a href="https://colab.research.google.com/github/sergiolitwiniuk85/Dart/blob/main/variant_calling_genome_dart_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Llamado de Variantes Completo para *Ilex paraguariensis* desde archivos FASTQ.gz**

Este Jupyter Notebook describe un flujo de trabajo completo para la identificación de variantes genéticas (SNPs y pequeños INDELs)
en datos de secuenciación DART de *Ilex paraguariensis*, comenzando desde los archivos FASTQ.gz.
*Ilex paraguariensis* corresponde a un organismo diploide.
* Se seguirán las buenas prácticas recomendadas para el análisis genómico de plantas.

**Requisitos Previos:**

1.  **Instalación de Herramientas:**: para ello crearemos un ambiente virtual en conda
    
    *   **fastqwiper**: para la correccion inicil de los archivos fastq corruptos que ya he identificado previamente.
    *   **fastp**: Para el preprocesamiento de las lecturas FASTQ (trimming de calidad y eliminación de adaptadores). (Este no se menciona directamente en las fuentes para plantas, pero es una buena práctica común).
    *   **BWA (Burrows-Wheeler Aligner)**: Para alinear las lecturas de secuenciación al genoma de referencia.
    *   **SAMtools**: Para manipular archivos en formato SAM/BAM (formato de alineación).
    *   **Picard Tools**: Para el manejo de archivos BAM, incluyendo la marcación de duplicados.
    *   **GATK (Genome Analysis Toolkit) [link text](https://gatk.broadinstitute.org/hc/en-us/articles/360035889851--How-to-Install-and-use-Conda-for-GATK4)**: Para el llamado de variantes, recalibración de calidad base y genotipado conjunto.


2.  **Genoma de Referencia de *Ilex paraguariensis*:** Archivo del genoma de referencia de *Ilex paraguariensis* en formato FASTA `ilex_paraguariensis_ref.fasta`) y archivo de índice para BWA (generado previamente con `bwa index`).

2.  **Transcriptoma de Referencia de *Ilex paraguariensis*:** Archivo del genoma de referencia de *Ilex paraguariensis* en formato FASTA.

3.  **Archivo de Nombres de Muestras:** Un archivo de texto (como "archivos\_muestras\_ilex\_paraguariensis - Hoja1.txt") que contenga la lista de nombres de tus archivos FASTQ.gz. Los archivos secuenciados provienen de una secuenciacion single-end.

**Configuración de Rutas y Variables:**




Antes de comenzar nos aseguramos de tener un entorno con todas las dependencias funcionando, para eso creo al archivo environment.yml con nano, y agrego los siguientes requerimientos:
```
name: dartcalling
channels:
  - conda-forge
  - bioconda
  - defaults
  - bfxcss
dependencies:
  - bwa-mem2  # Una versión más moderna y recomendada de BWA
  - samtools
  - picard  # Picard suele instalarse como un paquete
  - gatk4   # La última versión de GATK
  - fastp
  - python=3.x  # Especifica la versión de Python que necesitas (ej: 3.9, 3.10)
  - fastqwiper
```

Creo el entorno nuevo:
```
conda env create -f environment.yml
```
Activo el entorno:
```
conda activate dartcalling
```




In [ ]:
"""
Crear grafico del pipeline,
con mermaid,
"""


'\nCrear grafico del pipeline,\ncon mermaid,\n'

In [ ]:
# -*- coding: utf-8 -*-


import os
import subprocess

# Define las rutas a las herramientas y archivos de referencia
bwa_path = "bwa"  # Asegúrate de que BWA esté en tu PATH o especifica la ruta completa
samtools_path = "samtools" # Asegúrate de que SAMtools esté en tu PATH o especifica la ruta completa
picard_path = "picard"    # Especifica la ruta al directorio de Picard Tools (ej: /path/to/picard.jar)
gatk_path = "gatk"        # Asegúrate de que GATK esté en tu PATH o especifica la ruta completa
fastp_path = "fastp"      # Asegúrate de que fastp esté en tu PATH o especifica la ruta completa

reference_genome = "ilex_paraguariensis_ref.fasta" # Reemplaza con la ruta a tu genoma de referencia
reference_index = "ilex_paraguariensis_ref.fasta.fai" # BWA genera índices con el mismo prefijo

fastq_dir = "./fastq_files" # Directorio donde se encuentran tus archivos FASTQ.gz
sample_list_file = "archivos_muestras_ilex_paraguariensis - Hoja1.txt"
output_dir = "./variant_calling_output"

# Asegúrate de que el directorio de salida exista
os.makedirs(output_dir, exist_ok=True)

# Lee la lista de nombres de muestra desde el archivo
try:
    with open(sample_list_file, 'r') as f:
        sample_names = [line.strip() for line in f]
except FileNotFoundError:
    print(f"Error: El archivo de lista de muestras '{sample_list_file}' no fue encontrado.")
    exit()

print("Lista de muestras a procesar:", sample_names)




SyntaxError: unterminated triple-quoted string literal (detected at line 289) (<ipython-input-2-c06bebcd53de>, line 289)

**Bucle Principal para el Procesamiento de Cada Muestra:**

El siguiente bucle iterará a través de cada nombre de muestra en la lista y ejecutará los pasos
de preprocesamiento, alineación, marcación de duplicados y llamado de variantes para cada muestra individualmente.


In [ ]:
for sample_name in sample_names:
    print(f"\n**Procesando muestra: {sample_name}**")

    # Define el nombre del archivo FASTQ para la muestra actual (Single-end)
    fastq = os.path.join(fastq_dir, f"{sample_name}.fastq.gz")

    if not os.path.exists(fastq):
        print(f"Advertencia: Archivo FASTQ no encontrado para la muestra '{sample_name}'. Saltando esta muestra.")
        continue

    # Define los nombres de los archivos de salida para esta muestra
    trimmed_fastq = os.path.join(output_dir, f"{sample_name}_trimmed.fastq.gz")
    sam_file = os.path.join(output_dir, f"{sample_name}.sam")
    bam_file = os.path.join(output_dir, f"{sample_name}.bam")
    sorted_bam_file = os.path.join(output_dir, f"{sample_name}_sorted.bam")
    marked_duplicates_bam = os.path.join(output_dir, f"{sample_name}_mkdup.bam")
    metrics_file = os.path.join(output_dir, f"{sample_name}_metrics.txt")
    gvcf_file = os.path.join(output_dir, f"{sample_name}.g.vcf.gz")

    """
    **Paso 1: Preprocesamiento de Lecturas FASTQ con fastp**

    Este paso realiza el trimming de calidad, eliminación de adaptadores y trimming de las primeras 19 bases.
    """
    print("  - Ejecutando fastp para trimming...")
    fastp_command = [
        fastp_path,
        "-i", fastq,
        "-o", trimmed_fastq,
        "--trim_front1", "19",
        "-q", "30",
        "-u", "20",
        "-n", "5",
        "--length_required", "50",
        "-h", os.path.join(output_dir, f"{sample_name}_fastp.html"),
        "-j", os.path.join(output_dir, f"{sample_name}_fastp.json")
    ]
    subprocess.run(fastp_command, check=True)




    """
    **Paso 2: Alineación de Lecturas al Genoma de Referencia con BWA-MEM**
    """
    print("  - Alineando lecturas con BWA-MEM...")
    bwa_mem_command = [
        bwa_path, "mem",
        "-t", "30",                   # Threads
        "-k", "19",                   # Minimum seed length (default)
        "-c", "50000",              # Increase max occurrences (vs default 10,000)
        "-R", f"@RG\\tID:{sample_name}\\tSM:{sample_name}",  # Read group
        reference_genome,
        trimmed_fastq
    ]
    with open(sam_file, "w") as outfile:
        subprocess.run(bwa_mem_command, stdout=outfile, check=True)








    """
    **Paso 3: Conversión de SAM a BAM y Ordenamiento con SAMtools**
    """
    print("  - Convirtiendo SAM a BAM y ordenando...")
    samtools_view_command = [samtools_path, "view", "-bS", sam_file]
    with open(bam_file, "wb") as outfile:
        subprocess.run(samtools_view_command, stdout=outfile, check=True)

    samtools_sort_command = [samtools_path, "sort", "-o", sorted_bam_file, bam_file]
    subprocess.run(samtools_sort_command, check=True)







    """
    **Paso 4: Indexación del Archivo BAM Ordenado con SAMtools**
    """
    print("  - Indexando archivo BAM ordenado...")
    samtools_index_command = [samtools_path, "index", sorted_bam_file]
    subprocess.run(samtools_index_command, check=True)







    """
    **Paso 5: Marcación de Duplicados PCR con Picard MarkDuplicates**
    """
    print("  - Marcando duplicados PCR con Picard...")
    mark_duplicates_command = [
        "java", "-Xmx8G", "-jar", picard_path, "MarkDuplicates",
        "-I", sorted_bam_file,
        "-O", marked_duplicates_bam,
        "-M", metrics_file,
        "--REMOVE_DUPLICATES", "false",          # Critical for SNP calling
        "--ASSUME_SORT_ORDER", "coordinate",     # Required for coordinate-sorted BAMs
        "--OPTICAL_DUPLICATE_PIXEL_DISTANCE", "100", # NovaSeq/HiSeqX settings(e.g., HiSeq 2000/2500, MiSeq, NextSeq 500) or a non-Illumina platform,
                                                      # you might need a different value (e.g., 100 is often default for older platforms,
                                                      # or even 0 to disable if read names don't contain coordinate info)
        "--CREATE_INDEX", "true",                # Required for GATK
        "--VALIDATION_STRINGENCY", "STRICT",     # Ensures BAM compliance
    ]
    subprocess.run(mark_duplicates_command, check=True)






    """
    **Paso 6: Indexación del Archivo BAM con Duplicados Marcados con SAMtools**
    """
    print("  - Indexando archivo BAM con duplicados marcados...")
    samtools_index_command = [samtools_path, "index", marked_duplicates_bam]
    subprocess.run(samtools_index_command, check=True)







    """
    **Paso 7: Llamado de Variantes con GATK HaplotypeCaller en Modo GVCF**
    """
    print("  - Llamando variantes con GATK HaplotypeCaller (modo GVCF)...")

    haplotype_caller_command = [
    gatk_path, "HaplotypeCaller",
    "-R", reference_genome,                      # Reference genome (must be indexed)
    "-I", marked_duplicates_bam,                 # Input BAM (marked duplicates)
    "-O", gvcf_file,                             # Output GVCF
    "-ERC", "GVCF",                              # Enable GVCF output
    "--ploidy", "2",                             # Diploid organisms
    "--native-pair-hmm-threads", "8",            # Optimize CPU usage
    "--minimum-mapping-quality", "20",           # Filter low-confidence reads
    "--dont-use-soft-clipped-bases", "true",     # Avoid false positives
    "--standard-min-confidence-threshold-for-calling", "30",  # Min quality for variants
    "--max-reads-per-alignment-start", "500",     # Reduce noise in deep regions
    "--max-effective-depth", "2000",  # Prevents over-processing ultra-deep regions

    "--max-alternate-alleles", "3",              # Limit complex loci

    "--disable-read-filter", "NotDuplicateReadFilter",  # DArT markers may amplify duplicates
    "--native-pair-hmm-threads", "4",  # Fewer threads to prevent memory issues

    "--annotation-group", "StandardAnnotation",   # Standard annotations
    "--annotation", "MappingQualityRankSumTest",  # Additional QC
    "--annotation", "ReadPosRankSumTest",
    "--emit-ref-confidence", "GVCF",             # Required for joint genotyping
    "--bam-output", f"{sample_name}_realigned.bam",  # Optional: debug realignments
    ]
    subprocess.run(haplotype_caller_command, check=True)



NameError: name 'sample_names' is not defined

**Paso 8: Genotipado Conjunto con GATK GenotypeGVCFs**

Después de generar archivos GVCF para cada muestra individual, este paso realiza el genotipado conjunto.
Esto refina las llamadas de variantes al considerar la información de todas las muestras simultáneamente.

In [ ]:

print("\n**Realizando Genotipado Conjunto con GATK GenotypeGVCFs...**")

gvcf_files_for_joint_genotyping = [os.path.join(output_dir, f"{name}.g.vcf.gz") for name in sample_names if os.path.exists(os.path.join(output_dir, f"{name}.g.vcf.gz"))]

if not gvcf_files_for_joint_genotyping:
    print("Advertencia: No se encontraron archivos GVCF para realizar el genotipado conjunto.")
else:
   joint_genotyping_command = [
    "java", "-Xmx16G", "-jar", gatk_path, "GenotypeGVCFs",
    "-R", reference_genome,
    "-O", os.path.join(output_dir, "raw_variants.vcf.gz"),
    "--tmp-dir", "/path/to/large_tmp",  # Critical for large datasets
    "--allow-old-rms-mapping-quality-annotation-data",  # For compatibility
    "--standard-min-confidence-threshold-for-calling", "30",

    # DArT-specific adjustments:
    "--max-alternate-alleles", "6",  # Increased for potential multi-allelic sites
    "--include-non-variant-sites", "false",  # Skip reference sites (already in GVCFs)
    "--annotate-with-num-discovered-alleles",  # Helps with multi-sample analysis

    # Performance optimizations:
]

# Add all GVCFs to the command
for gvcf_file in gvcf_files_for_joint_genotyping:
    joint_genotyping_command.extend(["-V", gvcf_file])

subprocess.run(joint_genotyping_command, check=True)

    """
    **Paso 9: Filtrado Duro de Variantes con GATK VariantFiltration**

    Este paso aplica filtros básicos ("duros") basados en varias anotaciones de calidad de las variantes
    para eliminar posibles falsos positivos. Los umbrales de filtrado pueden necesitar ser ajustados
    dependiendo de las características específicas del conjunto de datos y el organismo.
    Los siguientes filtros son sugerencias basadas en prácticas comunes:

    *   Para SNPs:
        *   `QD < 2.0`: Calidad del sitio dividida por la profundidad de lectura.
        *   `FS > 60.0`: Fisher Strand, mide el sesgo de hebra.
        *   `MQ < 40.0`: Calidad de mapeo RMS.
        *   `SOR > 3.0`: StrandOddsRatio, otra medida de sesgo de hebra.

    *   Para INDELs:
        *   `QD < 2.0`
        *   `FS > 200.0`
        *   `ReadPosRankSum < -20.0`: Prueba de suma de rangos de posición de lectura.
    """
    print("\n**Aplicando Filtrado Duro de Variantes con GATK VariantFiltration...**")

    raw_variants_vcf = os.path.join(output_dir, "raw_variants.vcf.gz")
    filtered_snps_vcf = os.path.join(output_dir, "filtered_snps.vcf.gz")
    filtered_indels_vcf = os.path.join(output_dir, "filtered_indels.vcf.gz")
    final_filtered_vcf = os.path.join(output_dir, "final_variants.vcf.gz")

    # Filtrado de SNPs
    filter_snps_command = [
        gatk_path, "VariantFiltration",
        "-R", reference_genome,
        "-V", raw_variants_vcf,
        "-O", filtered_snps_vcf,
        "--filter-expression", "QD < 2.0 || FS > 60.0 || MQ < 40.0 || SOR > 3.0",
        "--filter-name", "snp_filter"
    ]
    subprocess.run(filter_snps_command, check=True)

    # Filtrado de INDELs
    filter_indels_command = [
        gatk_path, "VariantFiltration",
        "-R", reference_genome,
        "-V", filtered_snps_vcf,  # Se filtra sobre el resultado del filtrado de SNPs
        "-O", filtered_indels_vcf,
        "--filter-expression", "QD < 2.0 || FS > 200.0 || ReadPosRankSum < -20.0",
        "--filter-name", "indel_filter"
    ]
    subprocess.run(filter_indels_command, check=True)

    # Unir SNPs e INDELs filtrados (esto es una forma sencilla, puede haber mejores estrategias)
    # Se recomienda evaluar los resultados de forma separada antes de combinarlos.
    # Para este ejemplo, simplemente renombramos el archivo de INDELs filtrados como el resultado final.
    os.rename(filtered_indels_vcf, final_filtered_vcf)

    # Indexar el archivo VCF final
    index_vcf_command = [gatk_path, "IndexFeatureFile", "-F", final_filtered_vcf]
    subprocess.run(index_vcf_command, check=True)

    print(f"\n**Flujo de trabajo completado. El archivo VCF final se encuentra en: {final_filtered_vcf}**")

"""


**Consideraciones Adicionales y Buenas Prácticas para Genomas de Plantas:**

*   **Calidad del Genoma de Referencia:** La precisión del llamado de variantes depende en gran medida de la calidad y completitud del genoma de referencia.
*   **Regiones Repetitivas y Variaciones Estructurales (SVs):** Los genomas de plantas a menudo contienen grandes cantidades de secuencias repetitivas y variaciones estructurales. Los flujos de trabajo estándar de llamado de SNPs pueden tener dificultades en estas regiones. Se pueden requerir herramientas y estrategias específicas para la detección de SVs (como DELLY, Lumpy, Manta).
*   **Ploidy:** Se ha especificado `-ploidy 2` para *Ilex paraguariensis*.
*   **Recalibración de Calidad Base (BQSR):** En organismos no humanos sin conjuntos de variantes conocidos, se puede realizar un "bootstrap" para generar un conjunto de variantes de alta confianza para usar en la BQSR. Esto implicaría una ronda inicial de llamado de variantes, filtrado estricto de las variantes con mayor confianza y luego usar ese conjunto para la BQSR y un nuevo llamado de variantes. Esto podría mejorar la precisión de las llamadas de variantes.
*   **Filtrado Avanzado (Machine Learning):** La fuente menciona el uso de métodos de filtrado basados en aprendizaje automático, como GATK VariantRecalibrator. Estos métodos utilizan conjuntos de variantes conocidos de alta calidad para entrenar un modelo y luego aplicar filtros adaptativos. Si se dispone de un conjunto de variantes de alta confianza para *Ilex paraguariensis*, se podría considerar este enfoque para un filtrado más preciso.
*   **Pan-genomas:** Si existe un pan-genoma de *Ilex paraguariensis* o especies relacionadas, podría ser útil para detectar "diversidad oculta" debido a variaciones de presencia/ausencia (PAVs).
*   **Validación:** Las variantes identificadas deben validarse experimentalmente siempre que sea posible, especialmente aquellas de importancia biológica o clínica.

Este notebook proporciona un punto de partida sólido para el llamado de variantes. La optimización de los parámetros y la incorporación de pasos más avanzados como la BQSR y el filtrado basado en aprendizaje automático pueden mejorar aún más la precisión de los resultados.